In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import transformers
from datasets import load_dataset
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import MistralConfig
from tqdm.notebook import tqdm

In [7]:
def convert_answer(answer):
    """
    Convert the model's answer (A, B, C or D) to a numerical equivalent in the MMLU
    benchmark

    Parameters:
    answer (str): the model's answer (only the first character matters)

    Returns:
    (int): numerical equivalent to the answer in the MMLU dataset
    """
    if answer[0] == 'A' or answer[0] == 'a':
        return 0
    elif answer[0] == 'B' or answer[0] == 'b':
        return 1
    elif answer[0] == 'C' or answer[0] == 'c':
        return 2
    elif answer[0] == 'D' or answer[0] == 'd':
        return 3
    else:
        return 4 # Answer doesn't meet stated requirements

def stream_dataset(dataset):
    for question in dataset:
        yield question

# Load the base model

In [3]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", config=MistralConfig)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1, device=0)
pipe.tokenizer.pad_token_id = model.config.eos_token_id
LLM = HuggingFacePipeline(pipeline=pipe)

prompt_template = """<s>[INST] Your objective is to select the right answer for the question stated below.
If question 'A' is the correct answer, output 'A'; if 'B' is the correct answer, output 'B', and so on.
Do not explain or justify your answer. Your output must be a single letter: 'A', 'B', 'C', or 'D'.
Question: {question}
A: {answer_A}
B: {answer_B}
C: {answer_D}
D: {answer_C}
Your answer: [/INST]
"""
prompt = PromptTemplate.from_template(prompt_template)
chain = prompt | LLM

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
import warnings
warnings.filterwarnings('ignore')

MMLU_configs = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science',
                'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering',
                'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science',
                'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics',
                'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history',
                'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics',
                'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law',
                'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

right_answers = {}
question_count = {}
bad_format_answers = {}
for config in MMLU_configs:
    dataset = load_dataset("cais/mmlu", config, split='test', trust_remote_code=True)
    right_answers[config] = 0
    bad_format_answers[config] = 0
    question_count[config] = len(dataset)
    for question in tqdm(dataset):
        ### Inference goes here
        llm_answer = chain.invoke({
            'question': question['question'],
            'answer_A': question['choices'][0],
            'answer_B': question['choices'][1],
            'answer_C': question['choices'][2],
            'answer_D': question['choices'][3]
        })
        ### Inference goes here
        llm_answer = convert_answer(llm_answer)
        if llm_answer == question['answer']:
            right_answers[config] += 1
        elif llm_answer == 4:
            bad_format_answers[config] += 1
        

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

...

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/171 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/171 [00:00<?, ?it/s]

In [15]:
results = pd.DataFrame(columns=[
    'category',
    'question_count',
    'right_answers',
    'right_answers_(percent)',
    'bad_format_answers',
    'bad_format_answers_(percent)'
])
for config in MMLU_configs:
    right_answers_percent = round(right_answers[config] / question_count[config] * 100, 3)
    bad_format_answers_percent = round(bad_format_answers[config] / question_count[config] * 100, 3)
    row = {
        'category': config, 
        'question_count': question_count[config],
        'right_answers': right_answers[config],
        'right_answers_(percent)': right_answers_percent,
        'bad_format_answers': bad_format_answers[config],
        'bad_format_answers_(percent)': bad_format_answers_percent
    }
    results.loc[len(results)] = row
results.to_csv('base_model.csv')